### Joel Greenblat's Magic Formula 

In [1]:
# Importação das bibliotecas
import numpy as np
import pandas as pd
import requests

In [2]:
# Extrai o banco de dados da página 'Fundamentus'
url     = ('https://www.fundamentus.com.br/resultado.php')
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
res     = requests.get(url, headers=headers)
df      = pd.read_html(res.content)[0]
df.head(5)

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,POPR4,1017.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0,...,0.0,"8,66%","5,65%",108,"15,25%","19,93%",000,545.803.00000,82.0,"30,93%"
1,CSTB4,14769.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0,...,0.0,"40,85%","28,98%",260,"22,40%","20,11%",000,8.420.670.00000,14.0,"31,91%"
2,CFLU4,1.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0,...,0.0,"8,88%","10,72%",110,"17,68%","32,15%",000,60.351.00000,6.0,"8,14%"
3,PMET3,0.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0,...,0.0,"0,00%","0,00%",0,"0,00%","4,10%",000,-290.863.00000,0.0,"37,74%"
4,MNSA3,42.0,0.0,0.0,0.0,"0,00%",0,0.0,0.0,0,...,0.0,"-208,15%","-362,66%",363,"-13,50%","145,70%",000,-9.105.00000,-652.0,"-41,11%"


In [3]:
# Mostra as colunas do DataFrame
df.columns

Index(['Papel', 'Cotação', 'P/L', 'P/VP', 'PSR', 'Div.Yield', 'P/Ativo',
       'P/Cap.Giro', 'P/EBIT', 'P/Ativ Circ.Liq', 'EV/EBIT', 'EV/EBITDA',
       'Mrg Ebit', 'Mrg. Líq.', 'Liq. Corr.', 'ROIC', 'ROE', 'Liq.2meses',
       'Patrim. Líq', 'Dív.Brut/ Patrim.', 'Cresc. Rec.5a'],
      dtype='object')

In [4]:
# Converte coluna ROIC de valores string para float
df['ROIC'] = df['ROIC'].str.replace(',','.').str.rstrip('%').astype('float');

In [5]:
# RANK EV/EBITDA
df2 = df[['Papel','EV/EBITDA','ROIC']].sort_values(by="EV/EBITDA", ascending=True) # Sort Crescente
df2.head(10)

# Seleciona valores maiores que 0
selecao = df2[df2['EV/EBITDA'] > 0]

# Novo dataframe com seleção
df3 = selecao
df3['Rank_EV'] = range(1,df3.shape[0]+1) # insere os indices do Rank
df3

<ipython-input-5-4848f5f1c741>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Rank_EV'] = range(1,df3.shape[0]+1) # insere os indices do Rank


,Papel,EV/EBITDA,ROIC,Rank_EV
226,HETA4,1.01455,2.98,1
201,HETA3,1.17976,2.98,2
113,PRML3,1.64031,-1.96,3
280,CMMA4,3.00000,0.00,4
245,CBMA4,7.86290,-0.71,5
...,...,...,...,...
210,TEKA3,24648.00000,-0.33,640
178,PMET6,44157.00000,38.97,641
162,BCAL6,45265.00000,38.97,642
120,PMET5,50388.00000,38.97,643


In [6]:
# RANK ROIC
df4 = df3
df4 = df3.sort_values(by="ROIC", ascending=False) # Sort Decrescente

df4['Rank_ROIC'] = range(1,df4.shape[0]+1) # Insere os Indices do Rank

df4.head(5)

,Papel,EV/EBITDA,ROIC,Rank_EV,Rank_ROIC
355,BPHA3,143.0,401.01,28,1
687,MPLU3,728.0,129.57,298,2
374,AGEN33,361.0,87.67,109,3
588,ATOM3,805.0,82.04,324,4
559,CMET4,534.0,81.40,195,5


In [7]:
# RANK Magic
df5 = df4
df5['Rank_Soma'] = df4['Rank_ROIC'] + df4['Rank_EV'] # Somatório dos Ranks

df5 = df5.sort_values(by="Rank_Soma") # Sort dos Ranks

df5['Rank_Magic'] = range(1,df5.shape[0]+1) # Insere indices

df5.head(10)

,Papel,EV/EBITDA,ROIC,Rank_EV,Rank_ROIC,Rank_Soma,Rank_Magic
395,MTBR4,16.0,55.87,8,10,18,1
392,MTBR3,9.0,55.87,7,11,18,2
626,PSSA3,49.0,54.98,13,13,26,3
355,BPHA3,143.0,401.01,28,1,29,4
406,VALE5,91.0,44.97,17,20,37,5
496,DSUL3,127.0,48.91,22,16,38,6
473,CMIN3,221.0,81.03,55,6,61,7
409,ETER4,163.0,39.90,39,29,68,8
329,CSPC4,143.0,29.18,27,55,82,9
366,ASSM4,188.0,35.38,45,38,83,10


In [8]:
# Tabela contendo os setores de cada papel
df_setorial = pd.read_csv('setorial.csv')

df_setorial.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
2,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",DOMMO,DMMO,NaN
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
4,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PET MANGUINH,RPMG,NaN


In [9]:
# Remove linhas vazias (problema gerado na hora de crirar o csv e corrigido aqui)
df_setorial2 = df_setorial[df_setorial["SEGMENTO.1"].notna()]

df_setorial2.head(5)

,SETOR ECONÔMICO,SUBSETOR,SEGMENTO,EMPRESA,CÓDIGO,SEGMENTO.1
0,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",3R PETROLEUM,RRRP,NM
1,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",COSAN,CSAN,NM
3,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",ENAUTA PART,ENAT,NM
5,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS,PETR,N2
6,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",PETROBRAS BR,BRDT,NM


In [10]:
df6 = df5.copy() # Copia do df5
df6['Papel'] = df5['Papel'].str.rstrip('1234567890') # Pega somente Letras do papel
df6['Codigo'] = df5['Papel'].str.extract('(\d)', expand=True).astype('int') # Separa o Tipo/Código em outra coluna (ordinária, preferencial, etc..)
df6.head(5)

,Papel,EV/EBITDA,ROIC,Rank_EV,Rank_ROIC,Rank_Soma,Rank_Magic,Codigo
395,MTBR,16.0,55.87,8,10,18,1,4
392,MTBR,9.0,55.87,7,11,18,2,3
626,PSSA,49.0,54.98,13,13,26,3,3
355,BPHA,143.0,401.01,28,1,29,4,3
406,VALE,91.0,44.97,17,20,37,5,5


In [11]:
# Próximo passo: PUXAR O SETOR DO df_setorial2 com base no 'Papel' do df6